In [85]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [86]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [87]:
df = pd.read_csv('../datasets/adult.csv')

In [88]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [89]:
df.shape

(32561, 15)

In [90]:
bdf = BaseDatasetTransform(df,target='income')

In [91]:
df,categorical_features = bdf.fit_transform()


В колонке income нет пропущенных значений
-------------------------------------------
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['workclass', 'education', 'marital.status', 'occupation',
       'relationship', 'race', 'sex', 'native.country'],
      dtype='object')

Числовые колонки:
Index(['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss',
       'hours.per.week'],
      dtype='object')


In [92]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям
income
<=50K    24720
>50K      7841
Name: count, dtype: int64
Следует ли выполнить стратифицированное раздеение на обучающую и тестовую выборку? y/n
Разделение датасета выполнено успешно


In [103]:
from sklearn.utils.class_weight import compute_class_weight 
#is_unbalance=True
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))


class_counts = pd.Series(y_train).value_counts()
positive_class_count = class_counts[1]
negative_class_count = class_counts[0]
scale_pos_weight = negative_class_count / positive_class_count


lgbm_model = lgb.LGBMClassifier(verbose=-1,is_balanced=True)



columns_to_delete = []  


model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='binary',
    metric='binary_error'
)

C:\Users\dmitry\AppData\Local\Temp\ipykernel_26564\2278719576.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  positive_class_count = class_counts[1]
C:\Users\dmitry\AppData\Local\Temp\ipykernel_26564\2278719576.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  negative_class_count = class_counts[0]


In [104]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['workclass',
                                                                  'education',
                                                                  'marital.status',
                                                                  'occupation',
                                                                  'relationship',
                                                                  'race', 'sex',
                                                                  'native.country'],
                                            columns_to_delete=[],
                                            model=LGBMClassifier(is_balanced=True,
                                                                 metric='binary_error',
                                                                 objective='binary',
                                                                 verbose=-1)))

In [105]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,Metric,Score
0,accuracy,0.870611
1,precision,0.832676
2,recall,0.799814
3,f1,0.814134
